## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,2022-07-29 01:15:36,-23.1203,-134.9692,73.44,75,8,20.76,light rain
1,1,Hobart,AU,2022-07-29 01:15:04,-42.8794,147.3294,48.06,64,20,14.97,few clouds
2,2,Jamestown,US,2022-07-29 01:15:37,42.0970,-79.2353,76.59,54,0,8.05,clear sky
3,3,Guane,CU,2022-07-29 01:15:37,22.2025,-84.0875,80.73,74,77,4.97,broken clouds
4,4,Arraial Do Cabo,BR,2022-07-29 01:15:38,-22.9661,-42.0278,71.24,84,13,25.50,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        293
City           293
Country        293
Date           293
Lat            293
Lng            293
Max Temp       293
Humidity       293
Cloudiness     293
Wind Speed     293
Description    293
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,2022-07-29 01:15:36,-23.1203,-134.9692,73.44,75,8,20.76,light rain
2,2,Jamestown,US,2022-07-29 01:15:37,42.0970,-79.2353,76.59,54,0,8.05,clear sky
3,3,Guane,CU,2022-07-29 01:15:37,22.2025,-84.0875,80.73,74,77,4.97,broken clouds
4,4,Arraial Do Cabo,BR,2022-07-29 01:15:38,-22.9661,-42.0278,71.24,84,13,25.50,few clouds
9,9,Kapaa,US,2022-07-29 01:12:49,22.0752,-159.3190,84.18,77,75,8.05,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
662,662,Matara,LK,2022-07-29 01:34:57,5.9485,80.5353,77.85,88,100,3.40,overcast clouds
664,664,Mitikas,GR,2022-07-29 01:34:58,38.6699,20.9466,78.78,48,0,2.48,clear sky
665,665,Berezovyy,RU,2022-07-29 01:34:59,51.6667,135.7000,74.77,84,100,2.37,overcast clouds
669,669,Tungor,RU,2022-07-29 01:35:01,53.3808,142.9564,70.97,64,0,7.11,clear sky


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.44,light rain,-23.1203,-134.9692,
2,Jamestown,US,76.59,clear sky,42.0970,-79.2353,
3,Guane,CU,80.73,broken clouds,22.2025,-84.0875,
4,Arraial Do Cabo,BR,71.24,few clouds,-22.9661,-42.0278,
9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
10,Belmonte,BR,70.29,clear sky,-15.8631,-38.8828,
13,Sibu,MY,82.71,few clouds,2.3000,111.8167,
17,Nanpiao,CN,71.10,overcast clouds,41.0982,120.7479,
20,Butaritari,KI,82.06,overcast clouds,3.0707,172.7902,
21,Weligama,LK,77.43,overcast clouds,5.9667,80.4167,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.44,light rain,-23.1203,-134.9692,People ThankYou
2,Jamestown,US,76.59,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Guane,CU,80.73,broken clouds,22.2025,-84.0875,base de Campismo LosPortales de Guane
4,Arraial Do Cabo,BR,71.24,few clouds,-22.9661,-42.0278,Pousada Porto Praia
9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))